In [174]:
import pandas as pd
import numpy as np

In [175]:
adidas = pd.read_csv('adidas.csv')

In [176]:
adidas

,url,name,sku,selling_price,original_price,currency,availability,color,category,source,source_website,breadcrumbs,description,brand,images,country,language,average_rating,reviews_count,crawled_at
0,https://www.adidas.com/us/beach-shorts/FJ5089....,Beach Shorts,FJ5089,40,NaN,USD,InStock,Black,Clothing,adidas United States,https://www.adidas.com,Women/Clothing,Splashing in the surf. Making memories with yo...,adidas,"https://assets.adidas.com/images/w_600,f_auto,...",USA,en,4.5,35,2021-10-23 17:50:17.331255
1,https://www.adidas.com/us/five-ten-kestrel-lac...,Five Ten Kestrel Lace Mountain Bike Shoes,BC0770,150,NaN,USD,InStock,Grey,Shoes,adidas United States,https://www.adidas.com,Women/Shoes,Lace up and get after it. The Five Ten Kestrel...,adidas,"https://assets.adidas.com/images/w_600,f_auto,...",USA,en,4.8,4,2021-10-23 17:50:17.423830
2,https://www.adidas.com/us/mexico-away-jersey/G...,Mexico Away Jersey,GC7946,70,NaN,USD,InStock,White,Clothing,adidas United States,https://www.adidas.com,Kids/Clothing,"Clean and crisp, this adidas Mexico Away Jerse...",adidas,"https://assets.adidas.com/images/w_600,f_auto,...",USA,en,4.9,42,2021-10-23 17:50:17.530834
3,https://www.adidas.com/us/five-ten-hiangle-pro...,Five Ten Hiangle Pro Competition Climbing Shoes,FV4744,160,NaN,USD,InStock,Black,Shoes,adidas United States,https://www.adidas.com,Five Ten/Shoes,The Hiangle Pro takes on the classic shape of ...,adidas,"https://assets.adidas.com/images/w_600,f_auto,...",USA,en,3.7,7,2021-10-23 17:50:17.615054
4,https://www.adidas.com/us/mesh-broken-stripe-p...,Mesh Broken-Stripe Polo Shirt,GM0239,65,NaN,USD,InStock,Blue,Clothing,adidas United States,https://www.adidas.com,Men/Clothing,Step up to the tee relaxed. This adidas golf p...,adidas,"https://assets.adidas.com/images/w_600,f_auto,...",USA,en,4.7,11,2021-10-23 17:50:17.702680
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
840,https://www.adidas.com/us/supernova-plus-shoes...,Supernova+ Shoes,FX2858,72,$120,USD,InStock,White,Shoes,adidas United States,https://www.adidas.com,Women/Shoes,Take off. Touch down. Repeat. These adidas run...,adidas,"https://assets.adidas.com/images/w_600,f_auto,...",USA,en,4.3,151,2021-10-23 17:52:31.937924
841,https://www.adidas.com/us/choigo-shoes/H00667....,Choigo Shoes,H00667,70,$100,USD,InStock,White,Shoes,adidas United States,https://www.adidas.com,Women/Shoes,"If you want drama, the bold female track and f...",adidas,"https://assets.adidas.com/images/w_600,f_auto,...",USA,en,4.7,135,2021-10-23 17:52:32.014973
842,https://www.adidas.com/us/daily-3.0-shoes/GZ77...,Daily 3.0 Shoes,GZ7705,35,$50,USD,InStock,Black,Shoes,adidas United States,https://www.adidas.com,Kids/Shoes,The style is in the details of the Daily 3.0 S...,adidas,"https://assets.adidas.com/images/w_600,f_auto,...",USA,en,4.7,190,2021-10-23 17:52:32.208426
843,https://www.adidas.com/us/daily-3.0-shoes/GZ77...,Daily 3.0 Shoes,GZ7706,40,$50,USD,InStock,Pink,Shoes,adidas United States,https://www.adidas.com,Kids/Shoes,The style is in the details of the Daily 3.0 S...,adidas,"https://assets.adidas.com/images/w_600,f_auto,...",USA,en,4.7,190,2021-10-23 17:52:32.293622


In [177]:
df1 = adidas

In [178]:
df1.columns

Index(['url', 'name', 'sku', 'selling_price', 'original_price', 'currency',
       'availability', 'color', 'category', 'source', 'source_website',
       'breadcrumbs', 'description', 'brand', 'images', 'country', 'language',
       'average_rating', 'reviews_count', 'crawled_at'],
      dtype='object')

In [179]:
categoricals = ['url', 'name', 'sku', 'currency',
       'availability', 'color', 'category', 'source', 'source_website',
       'breadcrumbs', 'description', 'brand', 'images', 'country', 'language']

### Obteniendo las columnas que tienen más de un nivel

In [180]:
target = .98

valid_columns = []
columns_many_levels = []
columns_one_level = []

for c in categoricals:
    x = df1.loc[ : , [c]].drop_duplicates()
    uniquevalues = x.shape[0]
    
    if uniquevalues == 1:
        columns_one_level.append(c)
    elif (uniquevalues / adidas.shape[0]) > target:
        columns_many_levels.append(c)
    else:
        valid_columns.append(c)

### Obteniendo las columnas válidas

In [181]:
valid_columns

['name', 'availability', 'color', 'category', 'breadcrumbs', 'description']

### Eliminando las columnas de más de un nivel, que al menos uno de sus niveles represente un 98% o más

In [182]:
df1 = adidas
target = .98
columns_to_delete = []
for i_ in valid_columns:
    x = df1.groupby([i_], as_index=False).agg({'sku' : ['count']})
    x.columns = x.columns.droplevel(1)
    x['condition'] = x.apply(lambda r: r['sku'] / df1.shape[0], axis=1)
    if((x['condition'].max())>=target):
        columns_to_delete.append(i_) 
    else:
        pass

for x_ in columns_to_delete:
    if(x_ in (valid_columns)):
       valid_columns.remove(x_)
    else:
        pass
    

### Mostrando las nuevas columnas válidas

In [183]:
valid_columns

['name', 'color', 'category', 'breadcrumbs', 'description']

### Obteniendo las columnas que tengan entre 3 y 6 niveles y la frecuencia de cada uno de sus niveles

In [184]:
df2 = adidas
y = pd.DataFrame()
target1 = 2
target2 = 7

valid_columns_2 = []

for c_ in valid_columns:
    x = df2.loc[ : , [c_]].drop_duplicates()
    uniquevalues = x.shape[0]
    
    x = df2.groupby([c_], as_index=False).agg({'sku' : ['count']})
    x.columns = x.columns.droplevel(1)
    uniquevalues = x.shape[0]
        
    if((uniquevalues<target2) & (uniquevalues>target1)):
        valid_columns_2.append(c_)
        column_selected = x.loc[: , [c_, 'sku']]

column_selected.columns = ['category','frecuencia']

column_selected = column_selected.sort_values(['frecuencia'], ascending=False)
column_selected

,category,frecuencia
2,Shoes,426
1,Clothing,337
0,Accessories,82


### Haciendo la validación para aplicar el valor Others al campo con menor frecuencia

In [185]:
column_selected['level'] = 'Others'
v_min = column_selected['frecuencia'].min()

column_selected['validation'] = column_selected.apply(lambda r: r['level'] if r['frecuencia'] == v_min else r['category'], axis=1)

column_selected

,category,frecuencia,level,validation
2,Shoes,426,Others,Shoes
1,Clothing,337,Others,Clothing
0,Accessories,82,Others,Others


### Obteneiendo el dataset para hacer merge

In [186]:
column_selected = column_selected.loc[ : , ['category','validation']]
column_selected

,category,validation
2,Shoes,Shoes
1,Clothing,Clothing
0,Accessories,Others


### Ejecutando merge

In [187]:
df2 = df2.merge(column_selected, on='category', how='left')

### Mostrando el dataset original con merge

In [188]:
df2

,url,name,sku,selling_price,original_price,currency,availability,color,category,source,...,breadcrumbs,description,brand,images,country,language,average_rating,reviews_count,crawled_at,validation
0,https://www.adidas.com/us/beach-shorts/FJ5089....,Beach Shorts,FJ5089,40,NaN,USD,InStock,Black,Clothing,adidas United States,...,Women/Clothing,Splashing in the surf. Making memories with yo...,adidas,"https://assets.adidas.com/images/w_600,f_auto,...",USA,en,4.5,35,2021-10-23 17:50:17.331255,Clothing
1,https://www.adidas.com/us/five-ten-kestrel-lac...,Five Ten Kestrel Lace Mountain Bike Shoes,BC0770,150,NaN,USD,InStock,Grey,Shoes,adidas United States,...,Women/Shoes,Lace up and get after it. The Five Ten Kestrel...,adidas,"https://assets.adidas.com/images/w_600,f_auto,...",USA,en,4.8,4,2021-10-23 17:50:17.423830,Shoes
2,https://www.adidas.com/us/mexico-away-jersey/G...,Mexico Away Jersey,GC7946,70,NaN,USD,InStock,White,Clothing,adidas United States,...,Kids/Clothing,"Clean and crisp, this adidas Mexico Away Jerse...",adidas,"https://assets.adidas.com/images/w_600,f_auto,...",USA,en,4.9,42,2021-10-23 17:50:17.530834,Clothing
3,https://www.adidas.com/us/five-ten-hiangle-pro...,Five Ten Hiangle Pro Competition Climbing Shoes,FV4744,160,NaN,USD,InStock,Black,Shoes,adidas United States,...,Five Ten/Shoes,The Hiangle Pro takes on the classic shape of ...,adidas,"https://assets.adidas.com/images/w_600,f_auto,...",USA,en,3.7,7,2021-10-23 17:50:17.615054,Shoes
4,https://www.adidas.com/us/mesh-broken-stripe-p...,Mesh Broken-Stripe Polo Shirt,GM0239,65,NaN,USD,InStock,Blue,Clothing,adidas United States,...,Men/Clothing,Step up to the tee relaxed. This adidas golf p...,adidas,"https://assets.adidas.com/images/w_600,f_auto,...",USA,en,4.7,11,2021-10-23 17:50:17.702680,Clothing
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
840,https://www.adidas.com/us/supernova-plus-shoes...,Supernova+ Shoes,FX2858,72,$120,USD,InStock,White,Shoes,adidas United States,...,Women/Shoes,Take off. Touch down. Repeat. These adidas run...,adidas,"https://assets.adidas.com/images/w_600,f_auto,...",USA,en,4.3,151,2021-10-23 17:52:31.937924,Shoes
841,https://www.adidas.com/us/choigo-shoes/H00667....,Choigo Shoes,H00667,70,$100,USD,InStock,White,Shoes,adidas United States,...,Women/Shoes,"If you want drama, the bold female track and f...",adidas,"https://assets.adidas.com/images/w_600,f_auto,...",USA,en,4.7,135,2021-10-23 17:52:32.014973,Shoes
842,https://www.adidas.com/us/daily-3.0-shoes/GZ77...,Daily 3.0 Shoes,GZ7705,35,$50,USD,InStock,Black,Shoes,adidas United States,...,Kids/Shoes,The style is in the details of the Daily 3.0 S...,adidas,"https://assets.adidas.com/images/w_600,f_auto,...",USA,en,4.7,190,2021-10-23 17:52:32.208426,Shoes
843,https://www.adidas.com/us/daily-3.0-shoes/GZ77...,Daily 3.0 Shoes,GZ7706,40,$50,USD,InStock,Pink,Shoes,adidas United States,...,Kids/Shoes,The style is in the details of the Daily 3.0 S...,adidas,"https://assets.adidas.com/images/w_600,f_auto,...",USA,en,4.7,190,2021-10-23 17:52:32.293622,Shoes


### Mostrando un ejemplo de artículo con la columna de merge

In [189]:
df2.loc[ df2['sku']=='B93219', : ]

,url,name,sku,selling_price,original_price,currency,availability,color,category,source,...,breadcrumbs,description,brand,images,country,language,average_rating,reviews_count,crawled_at,validation
11,https://www.adidas.com/us/athletic-cushioned-c...,Athletic Cushioned Crew Socks 6 Pairs,B93219,20,NaN,USD,InStock,White,Accessories,adidas United States,...,Men/Accessories,Stop searching for the lost match to your favo...,adidas,"https://assets.adidas.com/images/w_600,f_auto,...",USA,en,4.0,5,2021-10-23 17:50:19.329862,Others


### Ingeniero, aquí no recuerdo si debíamos reemplazar la columna que habíamos cambiado por la validación, si es así... Continuaría lo siguiente; sino es así, la solución queda hasta el punto anterior

In [190]:
df2 = df2.loc[ : , ['url', 'name', 'sku', 'selling_price', 'original_price', 'currency',
       'availability', 'color', 'validation', 'source', 'source_website',
       'breadcrumbs', 'description', 'brand', 'images', 'country', 'language',
       'average_rating', 'reviews_count', 'crawled_at']]

In [191]:
df2.columns = ['url', 'name', 'sku', 'selling_price', 'original_price', 'currency',
       'availability', 'color', 'category', 'source', 'source_website',
       'breadcrumbs', 'description', 'brand', 'images', 'country', 'language',
       'average_rating', 'reviews_count', 'crawled_at']

In [192]:
df2

,url,name,sku,selling_price,original_price,currency,availability,color,category,source,source_website,breadcrumbs,description,brand,images,country,language,average_rating,reviews_count,crawled_at
0,https://www.adidas.com/us/beach-shorts/FJ5089....,Beach Shorts,FJ5089,40,NaN,USD,InStock,Black,Clothing,adidas United States,https://www.adidas.com,Women/Clothing,Splashing in the surf. Making memories with yo...,adidas,"https://assets.adidas.com/images/w_600,f_auto,...",USA,en,4.5,35,2021-10-23 17:50:17.331255
1,https://www.adidas.com/us/five-ten-kestrel-lac...,Five Ten Kestrel Lace Mountain Bike Shoes,BC0770,150,NaN,USD,InStock,Grey,Shoes,adidas United States,https://www.adidas.com,Women/Shoes,Lace up and get after it. The Five Ten Kestrel...,adidas,"https://assets.adidas.com/images/w_600,f_auto,...",USA,en,4.8,4,2021-10-23 17:50:17.423830
2,https://www.adidas.com/us/mexico-away-jersey/G...,Mexico Away Jersey,GC7946,70,NaN,USD,InStock,White,Clothing,adidas United States,https://www.adidas.com,Kids/Clothing,"Clean and crisp, this adidas Mexico Away Jerse...",adidas,"https://assets.adidas.com/images/w_600,f_auto,...",USA,en,4.9,42,2021-10-23 17:50:17.530834
3,https://www.adidas.com/us/five-ten-hiangle-pro...,Five Ten Hiangle Pro Competition Climbing Shoes,FV4744,160,NaN,USD,InStock,Black,Shoes,adidas United States,https://www.adidas.com,Five Ten/Shoes,The Hiangle Pro takes on the classic shape of ...,adidas,"https://assets.adidas.com/images/w_600,f_auto,...",USA,en,3.7,7,2021-10-23 17:50:17.615054
4,https://www.adidas.com/us/mesh-broken-stripe-p...,Mesh Broken-Stripe Polo Shirt,GM0239,65,NaN,USD,InStock,Blue,Clothing,adidas United States,https://www.adidas.com,Men/Clothing,Step up to the tee relaxed. This adidas golf p...,adidas,"https://assets.adidas.com/images/w_600,f_auto,...",USA,en,4.7,11,2021-10-23 17:50:17.702680
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
840,https://www.adidas.com/us/supernova-plus-shoes...,Supernova+ Shoes,FX2858,72,$120,USD,InStock,White,Shoes,adidas United States,https://www.adidas.com,Women/Shoes,Take off. Touch down. Repeat. These adidas run...,adidas,"https://assets.adidas.com/images/w_600,f_auto,...",USA,en,4.3,151,2021-10-23 17:52:31.937924
841,https://www.adidas.com/us/choigo-shoes/H00667....,Choigo Shoes,H00667,70,$100,USD,InStock,White,Shoes,adidas United States,https://www.adidas.com,Women/Shoes,"If you want drama, the bold female track and f...",adidas,"https://assets.adidas.com/images/w_600,f_auto,...",USA,en,4.7,135,2021-10-23 17:52:32.014973
842,https://www.adidas.com/us/daily-3.0-shoes/GZ77...,Daily 3.0 Shoes,GZ7705,35,$50,USD,InStock,Black,Shoes,adidas United States,https://www.adidas.com,Kids/Shoes,The style is in the details of the Daily 3.0 S...,adidas,"https://assets.adidas.com/images/w_600,f_auto,...",USA,en,4.7,190,2021-10-23 17:52:32.208426
843,https://www.adidas.com/us/daily-3.0-shoes/GZ77...,Daily 3.0 Shoes,GZ7706,40,$50,USD,InStock,Pink,Shoes,adidas United States,https://www.adidas.com,Kids/Shoes,The style is in the details of the Daily 3.0 S...,adidas,"https://assets.adidas.com/images/w_600,f_auto,...",USA,en,4.7,190,2021-10-23 17:52:32.293622


In [193]:
df2.loc[df2['sku']=='B93219', : ]

,url,name,sku,selling_price,original_price,currency,availability,color,category,source,source_website,breadcrumbs,description,brand,images,country,language,average_rating,reviews_count,crawled_at
11,https://www.adidas.com/us/athletic-cushioned-c...,Athletic Cushioned Crew Socks 6 Pairs,B93219,20,NaN,USD,InStock,White,Others,adidas United States,https://www.adidas.com,Men/Accessories,Stop searching for the lost match to your favo...,adidas,"https://assets.adidas.com/images/w_600,f_auto,...",USA,en,4.0,5,2021-10-23 17:50:19.329862
